Installing dependencies

In [ ]:
#This wont run from jupyter. So run this on terminal
#sudo apt-get install python3 python3-dev python3-numpy python3-pip portaudio19-dev git

In [ ]:
!pip3 install amodem uuid

Importing necessary libraries

In [ ]:
import os
import uuid
from ipywidgets import FileUpload
import base64
from cryptography.fernet import Fernet

Setting bitrate for 2kbps

In [ ]:
def setBitrate():
    os.environ['BITRATE']='16'

In [ ]:
setBitrate()

 Calibration

Check audio output device

In [ ]:
!amodem send --calibrate

Interrupt the kernel when calibration is done

In [ ]:
#Variable for filename
global fln
fln=''

Handle upload

In [ ]:
def on_upload_change(change):
    global fln
    if not change.new:
        return
    up = change.owner
    for filename,data in up.value.items():
        with open(filename, 'wb') as f:
            fln=filename
            f.write(data['content'])
    up.value.clear()
    up._counter = 0

upload_btn = FileUpload()
upload_btn.observe(on_upload_change, names='_counter')
upload_btn


In [ ]:
print("Filename: "+fln)

In [ ]:
file = open(fln, 'rb')
file_content = file.read()
file.close()

Generate metadata: Filename and its length

In [ ]:
flnlen=len(fln).to_bytes(1,byteorder='big')
metadata=flnlen+fln.encode()
data=metadata+file_content

In [ ]:
data

In [ ]:
tempdata=data

Combine metadata and data and write to a temporary file

In [ ]:
tempname=str(uuid.uuid4())
with open(tempname, 'wb') as f:
    f.write(tempdata)
os.environ['tempname']=tempname

In [ ]:
tempname

In [ ]:
tempsize = os.path.getsize(tempname)
print(f"Attempting to send {tempsize} bytes")
print("Make sure receiver is listening")

Sending

In [ ]:
!echo $BITRATE

In [ ]:
!amodem send -vv -i $tempname

Cleanup: Delete temporary file and uploaded file

In [ ]:
os.remove(fln)
os.remove(tempname)